# Import Libraries

In [ ]:

import gurobipy as grb
import pandas as pd
from gurobipy import GRB
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.3 MB/s eta 0:00:00


# Model

In [ ]:
# Create a model
model = grb.Model("Model")

Restricted license - for non-production use only - expires 2024-10-28


# Get data from json

In [ ]:
# get the  data from the json file small.json
with open('toy_instance.json') as f:
    data = json.load(f)

horizon=data['horizon']
qualifications=data['qualifications']
staff_names=[i['name'] for i in data['staff']]
staff_qualifications=[i['qualifications'] for i in data['staff']]
job_list=[i['name'] for i in data['jobs']]
jour_list=[i for i in range(1,horizon+1)]
job_penality=[i['daily_penalty'] for i in data['jobs']] 
job_qualifications= [i['working_days_per_qualification'] for i in data['jobs']]
job_gain=[i['gain'] for i in data['jobs']]
due_dates=[i['due_date'] for i in data['jobs']]
vocation_staff=[i['vacations'] for i in data['staff']]

# Model's parameters

In [ ]:
# create a function that converts a dictionary to a matrix
def get_matrix(liste):
    test=pd.DataFrame.from_dict(liste,orient='index')
    test.fillna(0,inplace=True)
    return test.to_dict(orient='index')

In [ ]:
# create a dictionary of with job_list as key and job_penality as value
gammak=job_penality_dict=dict(zip(job_list,job_penality))
gk=job_gain_dict=dict(zip(job_list,job_gain))
lk=due_dates_dict=dict(zip(job_list,due_dates))

pi_c_df=pd.DataFrame.from_dict(dict(zip(staff_names,[i['qualifications'] for i in data['staff']])),
                                orient='index').fillna(0)
pi_c_df.columns=qualifications
# convert string values to 1
pi_c_df=pi_c_df.applymap(lambda x: 1 if x in qualifications else 0)
pi_c=pi_c_df.to_dict(orient='index')

qk_c=get_matrix(dict(zip(job_list,job_qualifications)))
# get pi_j
staff_in_vacation=pd.DataFrame(index=staff_names,columns=jour_list)
# fill the dataframe staff_in_vacation with the svalues of the dictionary
for i in staff_names:
    for j in jour_list:
        staff_in_vacation.loc[i,j]=1 if j in dict(zip(staff_names,vocation_staff))[i] else 0
pi_j=staff_in_vacation.to_dict(orient='index')


# Decision variables

In [ ]:
# Add decision variables
# pi_k_j_c: Le personnel i est affecté au projet k sur la compétence c pour le jour j
pi_k_j_c = model.addVars(staff_names,job_list,jour_list,qualifications, vtype=GRB.BINARY, name="pi_k_j_c")
# pk_j : 1 si on travaille sur le projet k le jour j sinon 0
pk_j = model.addVars(job_list,jour_list,vtype=GRB.BINARY, name="pk_j")
# rk : 1 si le projet k est réalisé 0 sinon
rk = model.addVars( job_list,vtype=GRB.BINARY, name="rk")
# pi_k : 1 si le personnel i est affecté au projet k 0 sinon
pi_k = model.addVars( staff_names,job_list,vtype=GRB.BINARY, name="rk")

model.update()

# Constraints

In [ ]:
#Un personnel i peut être associé au projet k que s’il possède la compétence c nécessaire pour le projet :
constr1={'pi_k_j_c':model.addConstr(pi_k_j_c[i,k,j,c]  + (1-pi_c[i][c]) <=1, name="contrainte1")
    for k,j in zip(job_list,jour_list) 
    for i,c in zip(staff_names,qualifications) }
constr2={'pi_k_j_c':model.addConstr(grb.quicksum(pi_k_j_c[i,k,j,c] for k,c in zip(job_list,qualifications)) <=1, name="contrainte2")
    for i,j in zip(staff_names,jour_list)}
constr3={'pi_k_j_c':model.addConstr(pi_k_j_c[i,k,j,c]  + (1-pi_j[i][j]) <=1, name="contrainte3")
    for k,j in zip(job_list,jour_list) 
    for i,c in zip(staff_names,qualifications) }
 
# constr4={}

In [ ]:
gk

{'Job1': 20, 'Job2': 15, 'Job3': 15, 'Job4': 20, 'Job5': 10}

# Objective functions

In [ ]:
model.setObjective(sum([gk[k]*rk[k] for k in job_list]), GRB.MAXIMIZE)

In [ ]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 66 rows, 540 columns and 78 nonzeros
Model fingerprint: 0x3576e7a0
Variable types: 0 continuous, 540 integer (540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 80.0000000

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 80 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.000000000000e+01, best bound 8.000000000000e+01, gap 0.0000%


In [ ]:
all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

for name, val in zip(names, values):
    print(f"{name} = {val}")

pi_k_j_c[Olivia,Job1,1,A] = -0.0
pi_k_j_c[Olivia,Job1,1,B] = -0.0
pi_k_j_c[Olivia,Job1,1,C] = -0.0
pi_k_j_c[Olivia,Job1,2,A] = -0.0
pi_k_j_c[Olivia,Job1,2,B] = -0.0
pi_k_j_c[Olivia,Job1,2,C] = -0.0
pi_k_j_c[Olivia,Job1,3,A] = -0.0
pi_k_j_c[Olivia,Job1,3,B] = -0.0
pi_k_j_c[Olivia,Job1,3,C] = -0.0
pi_k_j_c[Olivia,Job1,4,A] = -0.0
pi_k_j_c[Olivia,Job1,4,B] = -0.0
pi_k_j_c[Olivia,Job1,4,C] = -0.0
pi_k_j_c[Olivia,Job1,5,A] = -0.0
pi_k_j_c[Olivia,Job1,5,B] = -0.0
pi_k_j_c[Olivia,Job1,5,C] = -0.0
pi_k_j_c[Olivia,Job2,1,A] = -0.0
pi_k_j_c[Olivia,Job2,1,B] = -0.0
pi_k_j_c[Olivia,Job2,1,C] = -0.0
pi_k_j_c[Olivia,Job2,2,A] = -0.0
pi_k_j_c[Olivia,Job2,2,B] = -0.0
pi_k_j_c[Olivia,Job2,2,C] = -0.0
pi_k_j_c[Olivia,Job2,3,A] = -0.0
pi_k_j_c[Olivia,Job2,3,B] = -0.0
pi_k_j_c[Olivia,Job2,3,C] = -0.0
pi_k_j_c[Olivia,Job2,4,A] = -0.0
pi_k_j_c[Olivia,Job2,4,B] = -0.0
pi_k_j_c[Olivia,Job2,4,C] = -0.0
pi_k_j_c[Olivia,Job2,5,A] = -0.0
pi_k_j_c[Olivia,Job2,5,B] = -0.0
pi_k_j_c[Olivia,Job2,5,C] = -0.0
pi_k_j_c[O